In [1]:
from pathlib import Path

In [2]:
# Updated imports to support OCR configuration
from docling.datamodel.document import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import ConversionStatus
from docling.datamodel.pipeline_options import PdfPipelineOptions

/home/fliperbaker/miniconda3/envs/rag1-mini/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
notebook_dir = Path.cwd()
PROJECT_ROOT = notebook_dir.parent

In [4]:
print(f"Notebook is in: {notebook_dir}")
print(f"Project root is: {PROJECT_ROOT}")

Notebook is in: /home/fliperbaker/projects/rag1/rag1-mini/notebooks
Project root is: /home/fliperbaker/projects/rag1/rag1-mini


In [5]:
source_file = PROJECT_ROOT / "data" / "raw" / "neuroscience" / "PRECLEAN_Brain_and_behavior_a_cognitive_neuroscience_perspective_David_Eagleman_Jonathan_Downar.pdf"
destination_file = PROJECT_ROOT / "data" / "processed" / "neuroscience" / "PRECLEAN_Brain_and_behavior_a_cognitive_neuroscience_perspective_David_Eagleman_Jonathan_Downar.md"

## conversion no_ocr no_tables

In [6]:
# 1. Define Pipeline Options
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False           # <--- Force OCR OFF (Fast & clean for digital docs)
pipeline_options.do_table_structure = False # Keep table recognition ON

In [7]:
# 2. Initialize Converter with specific PDF options
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

In [8]:
result = converter.convert(source_file)

2025-12-03 05:26:16,501 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-03 05:26:16,783 - INFO - Going to convert document batch...
2025-12-03 05:26:16,784 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 1216607fb7e04989285a12764e030fc9
2025-12-03 05:26:16,792 - INFO - Loading plugin 'docling_defaults'
2025-12-03 05:26:16,794 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-03 05:26:16,800 - INFO - Loading plugin 'docling_defaults'
2025-12-03 05:26:16,805 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-03 05:26:16,812 - INFO - Accelerator device: 'cpu'
2025-12-03 05:26:17,427 - INFO - Processing document PRECLEAN_Brain_and_behavior_a_cognitive_neuroscience_perspective_David_Eagleman_Jonathan_Downar.pdf
2025-12-03 05:37:11,163 - INFO - Finished converting document PRECLEAN_Brain_and_behavior_a_cognitive_neuroscience_perspective_David_Eagleman_Jonathan_Downar.pdf in 654.66 

In [11]:
# Access the internal DoclingDocument object
doc = result.document

## save markdown and json 

In [ ]:
destination_file.write_text(result.document.export_to_markdown(), encoding="utf-8")

In [ ]:
destination_json = PROJECT_ROOT / "data" / "processed" / "neuroscience" / "PRECLEAN_Brain_and_behavior.json"

In [ ]:
# 4. Option B: Save to a JSON file for easier inspection in a text editor
doc.save_as_json(destination_json)

## possible labels of docling items
https://github.com/docling-project/docling-core/blob/main/docling_core/types/doc/labels.py
```
class DocItemLabel(str, Enum):
    """DocItemLabel."""

    CAPTION = "caption"
    CHART = "chart"
    FOOTNOTE = "footnote"
    FORMULA = "formula"
    LIST_ITEM = "list_item"
    PAGE_FOOTER = "page_footer"
    PAGE_HEADER = "page_header"
    PICTURE = "picture"
    SECTION_HEADER = "section_header"
    TABLE = "table"
    TEXT = "text"
    TITLE = "title"
    DOCUMENT_INDEX = "document_index"
    CODE = "code"
    CHECKBOX_SELECTED = "checkbox_selected"
    CHECKBOX_UNSELECTED = "checkbox_unselected"
    FORM = "form"
    KEY_VALUE_REGION = "key_value_region"
    GRADING_SCALE = "grading_scale"  # for elements in forms, questionaires representing a grading scale
    # e.g. [strongly disagree | ... | ... | strongly agree]
    # e.g. ★★☆☆☆
    HANDWRITTEN_TEXT = "handwritten_text"
    EMPTY_VALUE = "empty_value"  # used for empty value fields in fillable forms

    # Additional labels for markup-based formats (e.g. HTML, Word)
    PARAGRAPH = "paragraph"
    REFERENCE = "reference"```

## delete unwanted items

DocItemLabel.CAPTION, DocItemLabel.FOOTNOTE, 
                    DocItemLabel.PAGE_FOOTER, DocItemLabel.PAGE_HEADER, 
                    DocItemLabel.TABLE
                    

In [9]:
from docling.document_converter import DocumentConverter
from docling.datamodel.document import DocItemLabel

In [12]:
# 2. Identify items to remove (e.g., remove all Captions and Footnotes)
# We collect them in a list first to avoid modifying the tree while iterating
items_to_remove = []
labels_to_remove = {DocItemLabel.CAPTION, DocItemLabel.FOOTNOTE, 
                    DocItemLabel.PAGE_FOOTER, DocItemLabel.PAGE_HEADER, 
                    DocItemLabel.TABLE}

for item, level in doc.iterate_items():
    # Check if the item has a label and if it matches our target list
    if hasattr(item, "label") and item.label in labels_to_remove:
        items_to_remove.append(item)

# 3. Delete the items from the document
# This updates the document tree in-place
doc.delete_items(node_items=items_to_remove)

In [ ]:
destination_file_removeditems = PROJECT_ROOT / "data" / "processed" / "neuroscience" / "PRECLEAN_Brain_and_behavior_removeditems.md"

In [ ]:
# 4. Now 'doc' can be passed to your RAG pipeline (e.g., chunking or export)
destination_file_removeditems.write_text(doc.export_to_markdown(), encoding="utf-8")

In [ ]:
destination_file_json_removeditems = PROJECT_ROOT / "data" / "processed" / "neuroscience" / "PRECLEAN_Brain_and_behavior_removeditems.json"

In [ ]:
doc.save_as_json(destination_file_json_removeditems)

## delete pictures and its children
removes picture elements that contain texts inside the picture. No OCR is done but small texts like (a) (b) digital text inside the picture makes the converter to create picture items (empty) that contains small texts. This is to remove that noise

In [13]:
# 1. Helper function to find all children recursively
def get_all_descendants(item):
    """Recursively collect all children, grandchildren, etc."""
    descendants = []
    # Check if the item has children
    if hasattr(item, "children") and item.children:
        for child in item.children:
            descendants.append(child)
            # Recursively get children of the child
            descendants.extend(get_all_descendants(child))
    return descendants

In [14]:
# 3. Identify items to remove
items_to_remove = []   # Use a list instead of a set
seen_ids = set()       # Track IDs to avoid duplicates

for item, level in doc.iterate_items():
    # Check if it is a Picture
    if hasattr(item, "label") and item.label == DocItemLabel.PICTURE:
        
        # A. Add the Picture item itself (if not already added)
        if id(item) not in seen_ids:
            items_to_remove.append(item)
            seen_ids.add(id(item))
        
        # B. Get all children (captions, texts inside)
        children = get_all_descendants(item)
        
        for child in children:
            if id(child) not in seen_ids:
                items_to_remove.append(child)
                seen_ids.add(id(child))

# 4. Delete the items
if items_to_remove:
    print(f"Removing {len(items_to_remove)} items...")
    doc.delete_items(node_items=items_to_remove)

Removing 7404 items...


In [15]:
destination_file_json_removedpictures= PROJECT_ROOT / "data" / "processed" / "neuroscience" / "PRECLEAN_Brain_and_behavior_removedpictures.json"

In [16]:
doc.save_as_json(destination_file_json_removedpictures)

In [17]:
destination_file_removedpictures= PROJECT_ROOT / "data" / "processed" / "neuroscience" / "PRECLEAN_Brain_and_behavior_removedpictures.md"

In [18]:
destination_file_removedpictures.write_text(doc.export_to_markdown(), encoding="utf-8")

2302755

## scispacy to sementation, cleaning and named entity recognition

In [19]:
import spacy
import re
from dataclasses import dataclass
from typing import List, Dict, Any

In [20]:
# --- CONFIGURATION & SETUP ---
# We define a data structure to keep our data clean as it moves through the pipeline
@dataclass
class ProcessedSentence:
    text: str                     # The actual sentence string
    entities: List[str]           # List of scientific entities found (e.g., "Hippocampus")
    entity_labels: List[str]      # The types of entities (e.g., "ORGAN", "CELL")
    source_metadata: Dict[str, Any] # The book info (Title, Author, Year)

In [21]:
class NeuroProcessor:
    def __init__(self):
        print("⏳ Loading SciSpacy models... (This happens only once)")
        
        # MODEL 1: Structural/Linguistic Model
        # Used for: Accurate Sentence Splitting
        try:
            self.nlp_seg = spacy.load("en_core_sci_md")
        except OSError:
            raise OSError("Could not load 'en_core_sci_md'. Did you install it?")

        # MODEL 2: Specialized NER Model
        # Used for: Extracting specific BioNLP entities
        try:
            self.nlp_ner = spacy.load("en_ner_bionlp13cg_md")
        except OSError:
            raise OSError("Could not load 'en_ner_bionlp13cg_md'. Did you install it?")
            
        print("✅ Models loaded successfully.")

    def process_text(self, raw_text: str, source_metadata: Dict[str, Any]) -> List[ProcessedSentence]:
        """
        Takes raw text and global metadata, returns a list of enriched sentence objects.
        """
        # 1. Basic Cleaning
        # PDF text often has line breaks in the middle of sentences. We flatten them.
        cleaned_text = re.sub(r'\s+', ' ', raw_text).strip()
        
        print(f"   -> Processing text length: {len(cleaned_text)} chars")

        # 2. Segmentation (Splitting into sentences)
        # We use the 'sci' model because it handles abbreviations (e.g., 'Fig. 1', 'et al.') better
        doc = self.nlp_seg(cleaned_text)
        
        # Filter: Ignore tiny sentences (< 10 chars) usually artifacts/page numbers
        sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 10]
        print(f"   -> Found {len(sentences)} valid sentences.")

        processed_output = []

        # 3. Enrichment (NER + Stamping)
        for sent in sentences:
            # Run the Specialized NER model on just this sentence
            ner_doc = self.nlp_ner(sent)
            
            # Extract unique entities
            entities = list(set([ent.text for ent in ner_doc.ents]))
            entity_types = list(set([ent.label_ for ent in ner_doc.ents]))
            
            # Create the object
            processed_sentence = ProcessedSentence(
                text=sent,
                entities=entities,
                entity_labels=entity_types,
                source_metadata=source_metadata # <--- Metadata Injection
            )
            processed_output.append(processed_sentence)
            
        return processed_output

In [22]:
# --- INITIALIZE PROCESSOR ---
processor = NeuroProcessor()

⏳ Loading SciSpacy models... (This happens only once)


/home/fliperbaker/miniconda3/envs/rag1-mini/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


✅ Models loaded successfully.
